In [32]:
print('Hello LangChain')

Hello LangChain


In [33]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print(OPENAI_API_KEY)

In [34]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("human", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="LangServe는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [35]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000002417877A810> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000024178778B30> root_client=<openai.OpenAI object at 0x000002417876F530> root_async_client=<openai.AsyncOpenAI object at 0x000002417877A5D0> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [36]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(response)
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
content='LangServe는 개발자가 대규모 언어 모델(LLM)을 쉽게 배포하고 관리할 수 있도록 도와주는 플랫폼입니다. LangServe를 사용하면 개발자는 언어 모델을 API로 감싸서 쉽게 사용할 수 있으며, 모델의 성능을 모니터링하고, 여러 모델을 관리하고, 사용량을 추적하는 등의 기능도 제공합니다.\n\nLangServe의 주요 기능은 다음과 같습니다:\n\n1. **모델 배포**: LangServe를 사용하면 개발자는 훈련된 언어 모델을 쉽게 배포할 수 있습니다. 모델을 API로 감싸서 다른 애플리케이션에서 쉽게 사용할 수 있습니다.\n\n2. **모델 관리**: LangServe는 여러 언어 모델을 관리할 수 있는 기능을 제공합니다. 개발자는 여러 모델을 등록하고, 각 모델의 성능을 모니터링하며, 필요에 따라 모델을 업데이트할 수 있습니다.\n\n3. **성능 모니터링**: LangServe는 모델의 성능을 실시간으로 모니터링할 수 있는 기능을 제공합니다. 개발자는 모델의 응답 시간, 정확도, 처리량 등의 지표를 확인하여 모델의 성능을 평가할 수 있습니다.\n\n4. **사용량 추적**: LangServe는 모델의 사용량을 추적할 수 있는 기능을 제공합니다. 개발자는 모델의 사용량, 사용자 정보, 요청 빈도 등의 지표를 확인하여 모델의 사용 패턴을 이해할 수 있습니다.\n\n5. **API 제공**: LangServe는 언어 모델을 API로 감싸서 다른 애플리케이션에서 쉽게 사용할 수 있도록 제공합니다. 개발자는 API를 사용하여 모델에 입력값을 제공하고, 모델의 출력을 받을 수 있습니다.\n\n6. **보안**: LangServe는 모델과 데이터를 안전하게 보호하기 위한 보안 기능을 제공합니다. 개발자는 모델과 데이터를 암호화하고, 접근을 제어하며, 감사 로그를 생성할 수 있습니다.\n\nLangServe를 사용하면 개발자는 언어 모델을 쉽게 배포하고 관리할

### LCEL 
* Prompt + LLM을 Chain으로 연결하기

In [37]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question. 
    <Question>: {input}에 대해 쉽게 설명해주세요.")
    """)                                     
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    You are an expert in AI Expert. Answer the question. \n    <Question>: {input}에 대해 쉽게 설명해주세요.")\n    ')

In [38]:
# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


### LCEL 
* Prompt + LLM + outputParser 를 Chain으로 연결하기

In [39]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL)
output_parser = StrOutputParser()
chain2 = prompt | llm | output_parser
print(type(chain2))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [40]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 방식과 유사합니다. 컴퓨터에大量的 데이터를 제공하고, 데이터를 분석하고 패턴을 찾아내도록 하는 것이죠.

예를 들어, 고양이와 강아지의 사진을 구분하는 인공지능 모델을 만든다고 가정해 봅시다. 

1. **데이터 수집**: 수많은 고양이와 강아지의 사진을 수집합니다.
2. **데이터 전처리**: 수집한 사진을 픽셀 단위로 나누어 숫자로 변환합니다. (예: 224 x 224 픽셀 = 50,176개의 숫자)
3. **모델 초기화**: 고양이와 강아지를 구분할 수 있는 모델을 만듭니다. (예: 신경망 모델)
4. **학습**: 수집한 데이터를 모델에 입력하고, 모델이 예측한 결과와 실제 결과를 비교합니다.
5. **오차 계산**: 예측 결과와 실제 결과의 차이를 계산합니다. (예: 예측 결과가 고양이인데 실제는 강아지인 경우)
6. **모델 업데이트**: 오차를 줄이기 위해 모델의 가중치를 업데이트합니다. (예: 고양이 사진의 픽셀 값이 강아지 사진의 픽셀 값과 다른 점을 학습)
7. **반복**: 4~6 단계를 반복하며 모델의 성능을 개선합니다.

이 과정을 통해 모델은 고양이와 강아지의 특징을 학습하고, 새로운 사진을 입력했을 때 고양이인지 강아지인지 구분할 수 있게 됩니다.

이러한 학습 원리는 다양한 인공지능 모델에 적용되며, 이를 통해 컴퓨터는 데이터를 분석하고 패턴을 찾아내어 문제를 해결할 수 있습니다.


In [41]:
# chain 호출 
try:
    result = chain2.invoke({"input": "LangChain의 Products(제품)는 어떤 것들이 있나요?"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 인공지능 모델을 활용하여 자연어 처리 및 생성 작업을 지원하는 플랫폼입니다. LangChain의 주요 제품은 다음과 같습니다.

1. **LangChain 모델**: LangChain은 다양한 자연어 처리 작업을 수행할 수 있는 사전 훈련된 언어 모델을 제공합니다. 이 모델은 텍스트 생성, 요약, 분류, 번역 등 다양한 작업에 사용할 수 있습니다.

2. **LangChain API**: LangChain API는 개발자가 LangChain 모델을 쉽게 사용할 수 있도록 지원하는 API입니다. 이 API를 통해 개발자는 자연어 처리 작업을 애플리케이션에 통합할 수 있습니다.

3. **LangChain Studio**: LangChain Studio는 LangChain 모델을 활용하여 자연어 처리 작업을 개발하고 테스트할 수 있는 통합 개발 환경입니다. 이 도구를 통해 개발자는 모델을 미세 조정하고, 작업 흐름을 구축하며, 결과를 시각화할 수 있습니다.

4. **LangChain Apps**: LangChain Apps는 LangChain 모델을 기반으로 구축된 다양한 애플리케이션입니다. 이 애플리케이션은 고객 서비스 챗봇, 콘텐츠 생성 도구, 언어 번역 서비스 등 다양한 분야에서 활용할 수 있습니다.

LangChain의 제품들은 자연어 처리 작업을 효율적으로 수행할 수 있도록 지원하며, 개발자와 기업이 인공지능 기술을 쉽게 활용할 수 있도록 돕습니다.


### Runnable의 stream() 메서드 호출
* invoke()는 한 번에 처리해서 반환
* stream()은 순차적으로 부분적인 결과(토큰)들을 반환

In [42]:
# chain 호출 
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세히 설명해줘"}) # 스트리밍 출력을 위한 요청
    # print(answer) # 스트리밍 출력
    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")


인공지능 모델의 학습 원리를 쉽게 설명해 드리겠습니다.

인공지능 모델의 학습 원리는 크게 다음과 같은 단계로 이루어져 있습니다.

1. **데이터 수집**: 인공지능 모델을 학습시키기 위해서는大量的 데이터가 필요합니다. 이 데이터는 문제에 대한 답이 포함된 형태여야 합니다. 예를 들어, 고양이와 강아지의 사진을 분류하는 모델을 만든다고 가정해 봅시다. 이 경우, 고양이와 강아지의 사진 데이터와 그에 대한 라벨(고양이 또는 강아지)이 필요합니다.

2. **데이터 전처리**: 수집된 데이터는 모델에 사용하기 전에 전처리 과정을 거칩니다. 데이터의 품질을 확인하고, 누락된 값이 있으면 채우고, 데이터의 크기를 조정하는 등의 작업을 수행합니다.

3. **모델 선택**: 인공지능 모델에는 여러 가지 종류가 있습니다. 예를 들어, 이미지 분류에는 CNN(Convolutional Neural Network), 자연어 처리에는 RNN(Recurrent Neural Network) 또는 Transformer 등이 있습니다. 적합한 모델을 선택해야 합니다.

4. **모델 학습**: 선택한 모델에 데이터를 입력하여 학습을 시작합니다. 모델은 입력된 데이터와 그에 대한 라벨을 비교하여 오류를 계산합니다. 이 오류를 최소화하는 방향으로 모델의 가중치를 조정하는 과정을 반복합니다.

5. **오류 최소화**: 모델의 예측 결과와 실제 라벨 간의 차이를 손실 함수(loss function)로 계산합니다. 이 손실 함수를 최소화하는 방향으로 모델의 파라미터를 업데이트합니다. 이 과정은 보통 경사 하강법(gradient descent) 알고리즘을 통해 수행됩니다.

6. **모델 평가**: 학습이 완료된 모델을 평가합니다. 평가 데이터에 모델을 적용하여 성능을 측정하고, 필요에 따라 모델을 수정하거나 학습을 재개합니다.

7. **모델 배포**: 최종적으로 모델을 배포하여 실제 문제에 적용합니다.

예를 들어, 고양이와 강아지의 사진을 분류하는 모델을 만든다고 가정해 봅시다. 모델에

### Multi Chain
* 첫번째 Chain 출력이 두번째 Chain의 입력이 된다
* 두개의 Chain과 Prompt + OutputParser 를 LCEL로 연결하기

In [43]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [44]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "액션"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

영화의 제목은 '다이 하드'입니다.

뉴욕의 나카토미 빌딩에서 크리스마스 파티가 열리는데, 테러리스트들이 빌딩을 점령하고 인질을 잡습니다. 주인공 존 맥클레인은 아내를 만나러 빌딩에 들어가게 되는데, 테러리스트들과 싸우게 됩니다. 액션과 스릴이 넘치는 이 영화는 많은 액션 영화의 영향을 받았고, 이후 많은 영화와 게임에서 패러디되었습니다.


### Prompt Template 여러 개 연결하기    

In [45]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})
print(response)

ChatGPT 모델의 학습 원리는 다음과 같습니다.

ChatGPT는 대규모의 텍스트 데이터를 기반으로 하는 언어 모델입니다. 이 모델은 주어진 문맥에 따라 다음에 올 단어를 예측하도록 학습하며, 이를 통해 자연스러운 대화가 가능하도록 설계되었습니다. 학습 과정에서 모델은 많은 양의 텍스트 데이터를 분석하고 패턴을 학습하여 대화의 맥락을 이해하고 적절한 응답을 생성합니다.

ChatGPT 모델의 장점은 다음과 같습니다.

* 자연스러운 대화가 가능
* 다양한 주제에 대한 지식 제공
* 창의적인 응답 생성 가능
* 사용자와의 상호작용을 통해 학습하고 개선 가능

ChatGPT 모델과 비슷한 AI 모델은 다음과 있습니다.

* LLaMA
* PaLM
* BERT
* RoBERTa
* transformer-XL 

등이 있습니다.


In [46]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "llama-4", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

['GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'llama-4 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']


In [47]:
# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt) #AIMessage
    print(response.content)

GPT-4 모델은 대규모의 텍스트 데이터를 학습하여 언어 패턴과 구조를 익히는 방식으로 훈련됩니다. 이 모델은 주어진 문맥에서 다음에 올 가능성이 높은 단어를 예측하도록 설계되었으며, 수백만 개의 매개변수를 조정하여 최적의 성능을 발휘하도록 합니다.
Gemma는 컴퓨터가 자연어 텍스트를 생성하거나 요약하거나 이해하는 것과 같은 자연어 처리 작업을 수행하도록 훈련된 경량 언어 모델입니다. Gemma는 대규모 언어 모델과 유사한 아키텍처와 훈련 데이터를 사용하여 상당한 성능 이점을 제공하지만 더 작은 버전으로 설계되어 더 많은 사람들이 액세스할 수 있습니다. 개발자와 연구원은 Gemma를 사용하여 자연어 처리 기능을 애플리케이션에 통합하고 언어 모델의 개발 및 개선을 가속화할 수 있습니다.
llama-4 모델은 메타에서 개발한 대규모 언어 모델입니다. 이 모델은 방대한 양의 텍스트 데이터를 학습하여 자연어 처리 능력을 키웁니다. 학습 과정에서 모델은 주어진 문맥에서 다음에 올 단어를 예측하도록 훈련되며, 이 과정을 통해 언어의 패턴과 구조를 학습합니다. 이를 통해 llama-4는 다양한 자연어 처리 작업에 활용될 수 있습니다.


ChatPromptTemplate
* SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate 사용

In [48]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "너는 {topic} 전문가야. 명확하고 자세히 설명해"
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="딥러닝이 뭐야?")

# LLM 호출
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

 

## Definition

딥러닝은 인공신경망을 사용하여 데이터를 학습하고 예측하는 머신러닝의 한 분야입니다. 딥러닝은 데이터의 패턴을 인식하고, 데이터를 분류하거나 예측하는 데 사용됩니다. 이러한 기술은 이미지 인식, 음성 인식, 자연어 처리 등 다양한 분야에서 사용됩니다.

### Key Concepts

* **인공신경망**: 딥러닝의 핵심 개념으로, 인간의 뇌를 모방한 신경망 구조를 컴퓨터에 구현한 것입니다. 인공신경망은 여러 층의 노드(뉴런)로 구성되어 있으며, 각 노드는 입력 데이터를 처리하여 출력을 생성합니다.
* **심층 학습**: 딥러닝은 인공신경망의 깊이를 증가시켜 학습의 정확도를 높이는 기술입니다. 심층 학습을 통해 인공신경망은 복잡한 패턴을 인식하고, 높은 수준의 추상화를 수행할 수 있습니다.
* **데이터 학습**: 딥러닝은 대량의 데이터를 사용하여 인공신경망을 학습시킵니다. 학습 과정에서 인공신경망은 데이터의 패턴을 인식하고, 모델을 업데이트하여 예측의 정확도를 높입니다.

### Applications

딥러닝은 다양한 분야에서 사용됩니다. 몇 가지 예시는 다음과 같습니다:

* **이미지 인식**: 딥러닝은 이미지 인식 분야에서 우수한 성능을 발휘합니다. 예를 들어, 얼굴 인식, 객체 탐지, 이미지 분류 등에 사용됩니다.
* **음성 인식**: 딥러닝은 음성 인식 분야에서 사용됩니다. 예를 들어, 음성 명령어 인식, 음성 번역 등에 사용됩니다.
* **자연어 처리**: 딥러닝은 자연어 처리 분야에서 사용됩니다. 예를 들어, 텍스트 분류, 감정 분석, 언어 번역 등에 사용됩니다.

### Advantages

딥러닝의 장점은 다음과 같습니다:

* **높은 정확도**: 딥러닝은 대량의 데이터를 사용하여 학습하기 때문에 높은 정확도를 달성할 수 있습니다.
* **복잡한 패턴 인식**: 딥러닝은 복잡한 패턴을 인식할 수 있습니다.
* **다양한 분야에 적용**: 딥러닝은 다양한 분야에 적용할 수 있습니다.

### Challenge

### FewShotPromptTemplate
* 예시 제공 프롬프트

In [49]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### 태양계의 행성
1. **수성**: 가장 작은 행성, 태양과 가장 가깝습니다.
2. **금성**: 매우 뜨겁고 밝은 행성입니다.
3. **지구**: 생명체가 사는 유일한 행성입니다.
4. **화성**: 붉은 행성으로, 로봇 탐사가 이루어지고 있습니다.
5. **목성**: 태양계에서 가장 큰 행성입니다.
6. **토성**: 아름다운 고리를 가진 행성입니다.
7. **천왕성**: 자전축이 기울어진 행성입니다.
8. **해왕성**: 태양계에서 가장 먼 행성입니다.


### PartialPromptTemplate
* 프롬프트 입력값에 동적인 메서드 호출이나 외부 API를 호출한 동적인 값을 대입 가능함

In [50]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month
    
    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)

# prompt = PromptTemplate(
#     template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}입니다.",
#     input_variables=["phenomenon"],  # 사용자 입력 필요
#     partial_variables={"season": get_current_season("south")}  # 동적으로 계절 값 할당
# )

season = get_current_season("south")
print(f"현재 계절: {season}")

prompt = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 초기화
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 특정 계절의 현상 질의
chain = (
    {"season": lambda x: season}
    | prompt
    | model
    | StrOutputParser()
)
result = chain.invoke({})


# 결과 출력
print(f"🔹 프롬프트: {prompt}")
print(f"🔹 모델 응답: {season}에 발생하는 자연 현상: \n {result}")

현재 계절: 겨울
🔹 프롬프트: input_variables=['season'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['season'], input_types={}, partial_variables={}, template='{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. 각 현상에 대해 간단한 설명을 포함해주세요.'), additional_kwargs={})]
🔹 모델 응답: 겨울에 발생하는 자연 현상: 
 1.  **극광 (Aurora)** 극광은 태양풍이 지구 자기장에 의해 극지방으로 빨려 들어가면서 대기 입자와 충돌하여 발생하는 현상입니다. 이 충돌로 인해 대기 입자가 흥분하고, 이 흥분된 입자들이 에너지를 방출하면서 빛을 발생시키게 됩니다. 이 빛은 녹색, 빨간색, 보라색 등 다양한 색상으로 나타나며, 밤하늘을 아름답게 수놓습니다. 극광은 북극지방에서는 북극광(Aurora Borealis), 남극지방에서는 남극광(Aurora Australis)이라고 불립니다.
2.  **빙하 (Glacier)** 빙하는 높은 고산 지역에서 눈이 쌓여 압축되어 형성된 거대한 얼음 덩어리입니다. 겨울에는 눈이 많이 쌓이고, 여름에는 눈이 녹으면서 빙하의 크기가 변합니다. 빙하는 지구의 기후 변화에 민감하게 반응하며, 빙하의 크기와 위치 변화는 지구온난화의 지표로 사용됩니다.
3.  **성층권 극소용돌이 (Stratospheric Polar Vortex)** 성층권 극소용돌이는 극지방의 성층권에서 발생하는 대규모 기상 현상입니다. 이 현상은 극지방의 공기가 극소용돌이로 인해 극지방에 갇히게 되고, 이로 인해 극지방의 기온이 급격히 떨어지게 됩니다. 이 현상은 북반구에서 주로 발생하며, 극지방의 기온을 낮추고, 중위도 지역의 기상 패턴에 영향을 미칩니다.


In [51]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD") # 환율 정보
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# {info} 변수에 API에서 받은 환율 정보를 동적으로 반영
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국 경제 미치는 영향 및 향후에 환율 분석을 제공해주세요. ",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

print(prompt)

input_variables=[] input_types={} partial_variables={'info': '1달러 = 1365.14원'} template='현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국 경제 미치는 영향 및 향후에 환율 분석을 제공해주세요. '


In [53]:
# # LLM 모델 설정 (GPT-4o-mini 사용)
# model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5 # 이번엔 0.5
)

print(f"prompt: {prompt}")

# 모델에 프롬프트 전달 및 응답 받기
response = model.invoke(prompt.format())

# 결과 출력
print("🔹 프롬프트:", prompt.format())
print("🔹 모델 응답:", response.content)

prompt: input_variables=[] input_types={} partial_variables={'info': '1달러 = 1365.14원'} template='현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국 경제 미치는 영향 및 향후에 환율 분석을 제공해주세요. '
🔹 프롬프트: 현재 1달러 = 1365.14원 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국 경제 미치는 영향 및 향후에 환율 분석을 제공해주세요. 
🔹 모델 응답: ## 한국 경제에 미치는 영향

### 1. 수출입에 미치는 영향

*   **수출 증가:** 약달러는 한국의 수출 경쟁력을 높여줍니다. 달러로 표시되는 수출 가격이 낮아지기 때문에, 해외 시장에서 한국 상품의 가격 경쟁력이 증가하여 수출량이 증가할 수 있습니다. 이는 자동차, 반도체, 철강 등 주요 수출 산업에 긍정적인 영향을 미칩니다.
*   **수입 비용 증가:** 반대로, 약달러는 수입 비용을 증가시킵니다. 수입 상품의 가격이 상승하기 때문에, 원유, 곡물, 원자재 등 수입에 의존하는 산업의 비용 부담이 증가할 수 있습니다.

### 2. 물가 상승

*   **인플레이션:** 약달러로 인해 수입 비용이 증가하면, 최종 소비재의 가격도 상승할 수 있습니다. 이는 물가 상승으로 이어질 수 있으며, 가계의 소비 부담을 증가시킬 수 있습니다.

### 3. 외국인 투자

*   **외국인 투자 감소:** 약달러는 외국인 투자 유치에 부정적인 영향을 미칠 수 있습니다. 한국 자산에 투자하는 외국인 투자자들은 환율 변동에 따른 손실을 우려할 수 있기 때문입니다.

### 4. 가계 부채

*   **가계 부채 부담 증가:** 약달러로 인해 가계 부채의 부담이 증가할 수 있습니다. 특히, 외화 대출을 받은 가계의 경우, 환율 상승으로 인해 부채 상환 부담이 증가할 수 있습니다.

## 향후 환율 전망

### 1. 글로벌 경제 상황

*   **미국 경제:** 미국 경제의 성장